In [1]:
import re 
import datetime
import pandas as pd

import urllib.request as urllib_request
from urllib.error import URLError, HTTPError

from utils.helpers import *
from utils.functions import *
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [87]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)

In [88]:
tickers = get_tickers(soup)

In [89]:
dataset = create_fiis_dataframe(tickers)

ABCP11
AFHI11
AGRX11
AIEC11
ALMI11
ALZM11
ALZR11
ANCR11B
APTO11
ARCT11
ARRI11
ASMT11
ATCR11
ATSA11
BARI11
BBFI11B
BBFO11
BBGO11
BBIM11
BBPO11
BBRC11
BCFF11
BCIA11
BCRI11
BICE11
BICR11
BIME11


HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset = break_tipo_FII(dataset)
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: str(convert_to_datetime(x)))
#dataset["p_vp"] = (dataset['cotacao'] / dataset['valor_patrimonial_por_cota']).replace(np.inf, np.nan)
dataset['date'] = str(datetime.date.today() - datetime.timedelta(days=1))
dataset.head()

NameError: name 'dataset' is not defined

In [95]:
URL = 'https://fiis.com.br/ifix/'
df_ifix = create_ifix_dataframe(URL)
COLUMNS_LIST = ['peso']
df_ifix = convert_percentages_to_float(df_ifix, COLUMNS_LIST)
df_ifix['peso'] = df_ifix['peso'].round(4)
df_ifix['date'] = str(datetime.date.today())
df_ifix.head()

,ticker,peso,date
0,KNIP11,0.0693,2022-09-30
1,KNCR11,0.0373,2022-09-30
2,HGLG11,0.0344,2022-09-30
3,IRDM11,0.0332,2022-09-30
4,KNRI11,0.0309,2022-09-30


In [40]:
df_ifix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  104 non-null    object 
 1   peso    104 non-null    float64
 2   date    104 non-null    object 
dtypes: float64(1), object(2)
memory usage: 2.6+ KB


## DynamoDB

### Sending the fiis dataframe

In [2]:
import boto3
import json
from decimal import Decimal

In [36]:
#ticker_data_dict = dataset.to_dict(orient='records')
ticker_data_dict = df.to_dict(orient='records')

In [4]:
dynamodb_resource = boto3.resource("dynamodb")

In [38]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict)

TypeError: Object of type Decimal is not JSON serializable

In [13]:
table = dynamodb_resource.Table("fiis")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.shape

(762, 22)

In [14]:
df.date.value_counts(dropna=False)

2022-09-29    381
2022-09-30    381
Name: date, dtype: int64

In [11]:
table = dynamodb_resource.Table("ifix")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.shape

(208, 3)

In [12]:
df.date.value_counts(dropna=False)

2022-09-30    104
2022-10-01    104
Name: date, dtype: int64

In [113]:
df.to_dict(orient='records')

[{'peso': Decimal('0.0088'), 'date': '2022-09-30', 'ticker': 'VGIP11'},
 {'peso': Decimal('0.0034'), 'date': '2022-09-30', 'ticker': 'RBRY11'},
 {'peso': Decimal('0.0084'), 'date': '2022-09-30', 'ticker': 'PVBI11'},
 {'peso': Decimal('0.0015'), 'date': '2022-09-30', 'ticker': 'HSAF11'},
 {'peso': Decimal('0.0026'), 'date': '2022-09-30', 'ticker': 'SADI11'},
 {'peso': Decimal('0.0029'), 'date': '2022-09-30', 'ticker': 'TEPP11'},
 {'peso': Decimal('0.0031'), 'date': '2022-09-30', 'ticker': 'VSLH11'},
 {'peso': Decimal('0.0056'), 'date': '2022-09-30', 'ticker': 'RECT11'},
 {'peso': Decimal('0.0062'), 'date': '2022-09-30', 'ticker': 'XPCI11'},
 {'peso': Decimal('0.0034'), 'date': '2022-09-30', 'ticker': 'KISU11'},
 {'peso': Decimal('0.0174'), 'date': '2022-09-30', 'ticker': 'HFOF11'},
 {'peso': Decimal('0.0145'), 'date': '2022-09-30', 'ticker': 'DEVA11'},
 {'peso': Decimal('0.0033'), 'date': '2022-09-30', 'ticker': 'PORD11'},
 {'peso': Decimal('0.0693'), 'date': '2022-09-30', 'ticker': 'KN

### Sending the ifix dataframe

In [96]:
dataset = df_ifix

In [84]:
dynamodb_resource = boto3.resource("dynamodb")
ifix_data_dict = dataset.to_dict(orient='records')
batch_write(dynamodb_resource, "ifix", ifix_data_dict)

True

In [105]:
table = dynamodb_resource.Table('ifix')

with table.batch_writer(overwrite_by_pkeys=['ticker', 'date']) as batch:
    for item in ifix_data_dict:
        #print(item)
        item = json.loads(json.dumps(item), parse_float=Decimal)
        batch.put_item(Item=item)